In [1]:
!pip install langchain
!pip install langchain_google_genai
!pip install langchain_community
!pip install chromadb
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67

In [2]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import Chroma

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [3]:
# @title CSV 파일 로드
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/Colab Notebooks/7_langchain/rag")

Mounted at /content/drive


In [4]:
loader = CSVLoader(
    file_path="./data/한국산업은행_금융 관련 용어_20151231.csv",
    encoding='cp949'
)
pages = loader.load()

print(pages[:2])


[Document(metadata={'source': './data/한국산업은행_금융 관련 용어_20151231.csv', 'row': 0}, page_content='구분: 리스크\n분류: 리스크 개요\n용어: 리스크(Risk)\n설명: 미래수익 또는 자산가치 변동의 불확실성(Uncertainty)으로 인하여 보유자산에서 손실이 발생할 가능성(신용  시장  금리  유동성리스크 등) 또한 부적절하거나 잘못된 내부절차  시스템 오류  직원의 실수·고의 또는 자연재해 등의 사건에 의해 손실이 발생할 가능성 (운영리스크 등)'), Document(metadata={'source': './data/한국산업은행_금융 관련 용어_20151231.csv', 'row': 1}, page_content='구분: 리스크\n분류: 리스크 개요\n용어: 불확실성\n설명: 설사 손실이 발생한다 해도 발생될 것이 확실하고 크기(금액)도 확실히 알 수 있어서 회피 또는 수용하기로 의사결정하고 나면 그것은 더 이상 리스크가 아님')]


In [5]:
# @title 임베딩 모델 로딩
hf_model = HuggingFaceEmbeddings(model_name='jhgan/ko-sroberta-multitask')

<ipython-input-5-4635b54cae4c>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  hf_model = HuggingFaceEmbeddings(model_name='jhgan/ko-sroberta-multitask')
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingfa

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
# @title 벡터 DB 생성
Chroma.from_documents(pages, hf_model, persist_directory="./database")

In [7]:
# @title 벡터 스토어 객체 로딩
hf_model = HuggingFaceEmbeddings(model_name='jhgan/ko-sroberta-multitask')
vector_store = Chroma(persist_directory="./database", embedding_function=hf_model)

<ipython-input-7-a445a3fb9899>:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vector_store = Chroma(persist_directory="./database", embedding_function=hf_model)


In [8]:
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [9]:
# @title MMR 검색기 생성
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={'k': 5, 'fetch_k': 10})

In [10]:
# @title 프롬프트 템플릿 생성
template = """
[context]: {context}
```
[질의]: {query}
```
[예시]
신용 환산율입니다.
```
위의 [context] 정보 내에서 [질의]에 대해 답변 [예시]와 같이 술어를 붙여서 답하세요.
"""
prompt = ChatPromptTemplate.from_template(template)

In [11]:
# @title 제미나이 객체 생성
from google.colab import userdata
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0, api_key=userdata.get('GOOGLE_API_KEY'))

In [12]:
# @title 검색 결과 가공하기
def merge_pages(pages):
    merged = "\n\n".join(page.page_content for page in pages)
    print(f"참조 문서 시작==>[\n{merged}\n]<==참조 문서 끝")
    return merged


In [13]:
# @title 체이닝 구성
chain = (
    {"query": RunnablePassthrough(), "context": retriever | merge_pages}
    | prompt
    | llm
    | StrOutputParser()
)


In [14]:
# @title 질의응답 테스트
answer = chain.invoke("짧은 기간 동안의 차익을 위해 금융사가 보유하는 걸 뜻하는 용어가 뭐야?")
print(f"answer1: {answer}\n\n")

answer = chain.invoke("트레이딩 포지션이 뭐야?")
print(f"answer2: {answer}")


참조 문서 시작==>[
구분: 리스크
분류: 시장리스크
용어: 트레이딩포지션
설명: 단기 매매차익 획득 등을 목적으로 금융기관이 보유하는 포지션으로 상품채권  상품주식 및 파생상품 등을 말함

구분: 리스크
분류: 시장리스크
용어: 트레이딩포지션
설명: 단기 매매차익 획득 등을 목적으로 금융기관이 보유하는 포지션으로 상품채권  상품주식 및 파생상품 등을 말함

구분: 퇴직연금
분류: 
용어: 단기금융시장
설명: 단기(통상 1년 미만) 금융자산을 거래하는 시장

구분: 리스크
분류: 리스크 개요
용어: 유동성리스크
설명: 자금운용과 조달기간의 불일치 또는 예기치 못한 자금유출 등으로 지급불능 상태에 직면하거나  자금의 과부족을 해소하기 위하여 비정상적 고금리 조달 또는 보유자산의 불리한 매각 등으로 손실이 발생할 가능성 (예금을 지급하기 위해 보유자산을 급하게 처분하는 과정에서 발생하는 손실 가능성 등)

구분: 퇴직연금
분류: 
용어: 정액분할투자
설명: 부담금을 한꺼번에 투자하지 않고 일정금액씩 주기적으로 나누어 평균 기준가격으로 투자함으로써 기준가격 변동의 불안정성을 일정 부분 Hedge하여 투자하는 기능으로  주로 거액을 일시에 납입할 경우 펀드로 자금을 한번에 투입시 그 시점의 시장의 상황에 수익률이 크게 좌우되는 불안정성을 완화하기 위해 개발된 기능
]<==참조 문서 끝
answer1: [예시] 트레이딩포지션입니다. 



참조 문서 시작==>[
구분: 리스크
분류: 시장리스크
용어: 트레이딩포지션
설명: 단기 매매차익 획득 등을 목적으로 금융기관이 보유하는 포지션으로 상품채권  상품주식 및 파생상품 등을 말함

구분: 리스크
분류: 시장리스크
용어: 트레이딩포지션
설명: 단기 매매차익 획득 등을 목적으로 금융기관이 보유하는 포지션으로 상품채권  상품주식 및 파생상품 등을 말함

구분: 리스크
분류: 시장리스크
용어: Front Office
설명: 트레이더(또는 딜러)가 고객 주문을 받아 대행 거래하거나 직접 자기 계산으로 유가증권ㆍ파생상품 등의 거래